In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 58.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=57584c35d2053a1f02ec6c82e0accadc4c46f504154f1cd6c90b42bbfce44d53
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("appName")
sc = SparkContext(conf=conf)

In [4]:
import operator
import re
import string

import math
from pyspark.sql.functions import *

# bow.hs en pyspark

In [10]:
def remove_useless_data(row):
    stop_words=[]
    pattern1 = re.compile(
        'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    )
    pattern2 = re.compile('[,.\"!@#$%^&*(){}?/;`~:<>+=-]')
    table = str.maketrans('', '', string.punctuation)

    row1 = row.lower()
    row2 = pattern1.sub('', row1)
    row3 = pattern2.sub('', row2)
    tokens = row3.split()
    row_no_punctuation = [w.translate(table) for w in tokens]
    row_no_num = [w for w in row_no_punctuation if w.isalpha()]
    row4 = [w for w in row_no_num if not w in stop_words]
    row = ' '.join(row4)
    return row 

In [7]:
data = sc.textFile("/content/corpus.txt")
data.collect()[:20]

['Zips this RDD with its element indices.',
 '',
 'The ordering is first based on the partition index and then the ordering of items within each partition. So the first item in the first partition gets index 0, and the last item in the last partition receives the largest index.',
 '',
 'This method needs to trigger a spark job when this RDD contains more than one partitions.']

# TF

In [11]:
data1 = data.flatMap(lambda line: line.split('\n')).map(lambda word: (word))
data1 = data1.zipWithIndex()
data2=data1.flatMap(lambda x: [((x[1] + 1,i)) for i in x[0].split('\n')])
data2 = data2.map(lambda x: (x[0], remove_useless_data(x[1])))
map1=data2.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
reduce=map1.reduceByKey(lambda x,y:x+y)
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()[:10]

[(1, 'zips this rdd with its element indices'), (2, ''), (3, 'the ordering is first based on the partition index and then the ordering of items within each partition so the first item in the first partition gets index and the last item in the last partition receives the largest index'), (4, ''), (5, 'this method needs to trigger a spark job when this rdd contains more than one partitions')]


[('zips', (1, 1)),
 ('rdd', (1, 1)),
 ('with', (1, 1)),
 ('its', (1, 1)),
 ('element', (1, 1)),
 ('the', (3, 8)),
 ('ordering', (3, 2)),
 ('first', (3, 3)),
 ('on', (3, 1)),
 ('index', (3, 3))]

# IDF

In [12]:
map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
lendata=data.count()
map4=map3.map(lambda x:(x[0],x[1][2]))
#Numero de terminos en el documento
reduce2=map4.reduceByKey(lambda x,y:x+y)
idf=reduce2.map(lambda x: (x[0],math.log10(lendata/x[1])))
idf.collect()[:10]

[('this', 0.3979400086720376),
 ('indices', 0.6989700043360189),
 ('is', 0.6989700043360189),
 ('based', 0.6989700043360189),
 ('partition', 0.6989700043360189),
 ('of', 0.6989700043360189),
 ('items', 0.6989700043360189),
 ('item', 0.6989700043360189),
 ('in', 0.6989700043360189),
 ('last', 0.6989700043360189)]

# TF-IDF

In [13]:
rdd=tf.join(idf)
tfidf=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
tfidf.collect()[:20]

[(1, ('this', 1, 0.3979400086720376, 0.3979400086720376)),
 (1, ('indices', 1, 0.6989700043360189, 0.6989700043360189)),
 (1, ('zips', 1, 0.6989700043360189, 0.6989700043360189)),
 (1, ('rdd', 1, 0.3979400086720376, 0.3979400086720376)),
 (1, ('with', 1, 0.6989700043360189, 0.6989700043360189)),
 (1, ('its', 1, 0.6989700043360189, 0.6989700043360189)),
 (1, ('element', 1, 0.6989700043360189, 0.6989700043360189)),
 (3, ('partition', 4, 0.6989700043360189, 2.7958800173440754)),
 (3, ('of', 1, 0.6989700043360189, 0.6989700043360189)),
 (3, ('items', 1, 0.6989700043360189, 0.6989700043360189)),
 (3, ('item', 2, 0.6989700043360189, 1.3979400086720377)),
 (3, ('in', 2, 0.6989700043360189, 1.3979400086720377)),
 (3, ('last', 2, 0.6989700043360189, 1.3979400086720377)),
 (3, ('the', 8, 0.6989700043360189, 5.591760034688151)),
 (3, ('first', 3, 0.6989700043360189, 2.0969100130080567)),
 (3, ('and', 2, 0.6989700043360189, 1.3979400086720377)),
 (3, ('then', 1, 0.6989700043360189, 0.6989700043360